# Tensor Network Message Passing

This tutorial is based on the julia package [GenericMessagePassing.jl](https://github.com/ArrogantGao/GenericMessagePassing.jl).

## Getting Started

Before runing the following code, please install the package by
```julia
pkg> add https://github.com/ArrogantGao/GenericMessagePassing.jl
```
and then install other necessary deps
```julia
pkg> instantiate
```

One can run the following code to reproduce the results the same as the python code, or directly run the script `494bus.jl` to get the results.
```julia
julia --project=. 494bus.jl
```


In [5]:
using GenericMessagePassing
using CSV, DataFrames

using Graphs, GraphIO
using TensorInference, OMEinsum


In [6]:
# load the 494 bus graph and the random J and h

g = loadgraph("data/494bus.dot")
df_J = CSV.read("data/494bus_J_random.csv", DataFrame)
df_h = CSV.read("data/494bus_h_random.csv", DataFrame)

Js_dict = Dict()
for row in eachrow(df_J)
    Js_dict[minmax(row.v1 + 1, row.v2 + 1)] = row.J
end

hs = df_h.h 
Js = Float64[]
for e in edges(g)
    v1, v2 = src(e), dst(e)
    push!(Js, Js_dict[minmax(v1, v2)])
end

In [7]:
# construct the tensor network model
tn, code, tensors = GenericMessagePassing.ising_model(g, -1 .* hs, -1 .* Js, 0.5);

# calculate the exact marginals using the tensor inference package
ti_sol = marginals(tn)

Dict{Vector{Int64}, Vector{Float64}} with 494 entries:
  [311] => [0.516055, 0.483945]
  [281] => [0.482438, 0.517562]
  [14]  => [0.474304, 0.525696]
  [464] => [0.457306, 0.542694]
  [491] => [0.520861, 0.479139]
  [386] => [0.528997, 0.471003]
  [433] => [0.455651, 0.544349]
  [447] => [0.431216, 0.568784]
  [314] => [0.502899, 0.497101]
  [357] => [0.442085, 0.557915]
  [461] => [0.490967, 0.509033]
  [480] => [0.589852, 0.410148]
  [334] => [0.508424, 0.491576]
  [403] => [0.529911, 0.470089]
  [174] => [0.455893, 0.544107]
  [322] => [0.509197, 0.490803]
  [269] => [0.484172, 0.515828]
  [315] => [0.497178, 0.502822]
  [123] => [0.562221, 0.437779]
  ⋮     => ⋮

In [11]:
tnbp_config = TNBPConfig(verbose = true, r = 7, optimizer = TreeSA(sc_target = 20, ntrials = 1, niters = 5, βs = 0.1:0.1:100), error = 1e-6);
tnbp_sol = marginal_tnbp(tn, tnbp_config);

--------------------------------
average size of neibs: 16.841666666666665
maximum size of neibs: 186
--------------------------------
maximum size of sc: 4.0
total contraction cost: 16.466936773190373
--------------------------------
iter 1: error_max = 0.8614038391391328
iter 2: error_max = 0.48326991419219956
iter 3: error_max = 0.3444205885019018
iter 4: error_max = 0.21850720233109744
iter 5: error_max = 0.1417604278927589
iter 6: error_max = 0.08267485898616572
iter 7: error_max = 0.07934906256293711
iter 8: error_max = 0.06053040482170069
iter 9: error_max = 0.03594556955877215
iter 10: error_max = 0.01824760249059898
iter 11: error_max = 0.01380004584722
iter 12: error_max = 0.011357386053667595
iter 13: error_max = 0.006397854554759996
iter 14: error_max = 0.004938978320702114
iter 15: error_max = 0.0033233134123270003
iter 16: error_max = 0.0019929466331080614
iter 17: error_max = 0.001705720322313753
iter 18: error_max = 0.0014884956526740045
iter 19: error_max = 0.000672505

In [12]:
total_err = 0.0
for i in keys(tnbp_sol)
    total_err += sum((ti_sol[[i]] - tnbp_sol[i]).^2)
end

total_err = sqrt(total_err) / 494

1.5302769002427566e-5